<a href="https://colab.research.google.com/github/assismauro/CursoPython/blob/master/Azimuth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Azimuth

This Colab Notebook calculates the crossing point of two lines of sight, plot the corresponding azimuths and show the crossing point distances.

To run:

 * Click "Run" button (small arrow) left to the next cell.
 * Fill data
 * Click "Run"button again
 * click in "Calculate!" button below the form cell.
 

In [5]:
!pip install simplekml
import ipywidgets as widgets
from IPython.display import display
from shapely.geometry import Point, LineString
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math
import folium
import simplekml

#@title {run: 'auto'} ## Dados de Entrada

#@markdown #### Torre A 
nomeA = "Ponto Ferruginoso" #@param {type:"string"}
xA = 603484.00 #@param {type:"number"}
yA = 7781714.00 #@param {type:"number"}
azA = 220 #@param {type:"number"}
 
#@markdown #### Torre B 
nomeB = "Posto 39" #@param {type:"string"}
xB = 600593.00 #@param {type:"number"}
yB = 7781310.00 #@param {type:"number"}
azB = 150 #@param {type:"number"}
 
button = widgets.Button(description="Calculate!")
output = widgets.Output()
 
# trigonometry
def sincos(azim):
    az = np.radians(azim)
    sinaz = np.sin(az)
    cosaz = np.cos(az)
    return sinaz,cosaz
 
def calcIntersection(xA,yA,azA,xB,yB,azB): 
    pointA = Point(xA,yA)
    sinA, cosA = sincos(azA)
    pointB = Point(xB,yB)
    sinB, cosB = sincos(azB)
    length = 10000
    endlineA = Point(pointA.x+(length*sinA), pointA.y+(length*cosA))
    endlineB = Point(pointB.x+(length*sinB), pointB.y+(length*cosB))
    line1 = LineString([pointA,endlineA])
    line2 = LineString([pointB,endlineB])
    intersection = LineString([pointB,endlineB]).intersection(LineString([pointA,endlineA]))
    return  ((pointA.distance(pointB),
             pointA.distance(intersection),
             pointB.distance(intersection)),
            intersection)
 
def plotResult(p1,p2,p3,nomeA,NomeB,distances):
     figure(num=None, figsize=(12, 9), dpi=80, facecolor='w', edgecolor='k')
     x_values = [p1[0], p3[0]]
     y_values = [p1[1], p3[1]]
     plt.plot(x_values, y_values,label='Visada '+nomeA)
     x_values = [p2[0], p3[0]]
     y_values = [p2[1], p3[1]]
     plt.plot(x_values, y_values,label='Visada '+nomeB)
     title = 'X: {:10.2f}, Y: {:10.2f}\nDistância entre torres: {:10.2f}m\n'.format(
             p3[0],p3[1],distances[0])+\
             'Distância {0} à interseção: {1:10.2f}m\n'.format(nomeA,distances[1])+\
             'Distância {0} à interseção: {1:10.2f}m\n'.format(nomeB,distances[2])
     plt.title(title)
     plt.legend()
     plt.show()

# converte utm para latlong
def utmToLatLng(zone, easting, northing, northernHemisphere=True):
    if not northernHemisphere:
        northing = 10000000 - northing

    a = 6378137
    e = 0.081819191
    e1sq = 0.006739497
    k0 = 0.9996

    arc = northing / k0
    mu = arc / (a * (1 - math.pow(e, 2) / 4.0 - 3 * math.pow(e, 4) / 64.0 - 5 * math.pow(e, 6) / 256.0))

    ei = (1 - math.pow((1 - e * e), (1 / 2.0))) / (1 + math.pow((1 - e * e), (1 / 2.0)))

    ca = 3 * ei / 2 - 27 * math.pow(ei, 3) / 32.0

    cb = 21 * math.pow(ei, 2) / 16 - 55 * math.pow(ei, 4) / 32
    cc = 151 * math.pow(ei, 3) / 96
    cd = 1097 * math.pow(ei, 4) / 512
    phi1 = mu + ca * math.sin(2 * mu) + cb * math.sin(4 * mu) + cc * math.sin(6 * mu) + cd * math.sin(8 * mu)

    n0 = a / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (1 / 2.0))

    r0 = a * (1 - e * e) / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (3 / 2.0))
    fact1 = n0 * math.tan(phi1) / r0

    _a1 = 500000 - easting
    dd0 = _a1 / (n0 * k0)
    fact2 = dd0 * dd0 / 2

    t0 = math.pow(math.tan(phi1), 2)
    Q0 = e1sq * math.pow(math.cos(phi1), 2)
    fact3 = (5 + 3 * t0 + 10 * Q0 - 4 * Q0 * Q0 - 9 * e1sq) * math.pow(dd0, 4) / 24

    fact4 = (61 + 90 * t0 + 298 * Q0 + 45 * t0 * t0 - 252 * e1sq - 3 * Q0 * Q0) * math.pow(dd0, 6) / 720

    lof1 = _a1 / (n0 * k0)
    lof2 = (1 + 2 * t0 + Q0) * math.pow(dd0, 3) / 6.0
    lof3 = (5 - 2 * Q0 + 28 * t0 - 3 * math.pow(Q0, 2) + 8 * e1sq + 24 * math.pow(t0, 2)) * math.pow(dd0, 5) / 120
    _a2 = (lof1 - lof2 + lof3) / math.cos(phi1)
    _a3 = _a2 * 180 / math.pi

    latitude = 180 * (phi1 - fact1 * (fact2 + fact3 + fact4)) / math.pi

    if not northernHemisphere:
        latitude = -latitude

    longitude = ((zone > 0) and (6 * zone - 183.0) or 3.0) - _a3

    return (latitude, longitude)

def plotMapFolium(z, p1, p2, p3, nomeA, nomeB, distances, northernHemisphere=False):
    lat, lng = utmToLatLng(z, p1[0], p1[1], northernHemisphere=False)
    g1 = (lat, lng)
    lat, lng = utmToLatLng(z, p2[0], p2[1], northernHemisphere=False)
    g2 = (lat, lng)
    lat, lng = utmToLatLng(z, p3[0], p3[1], northernHemisphere=False)
    g3 = (lat, lng)

    points = [g1, g2, g3]
    #print(points)

    # Load map centred on average coordinates
    ave_lat = sum(p[0] for p in points)/len(points)
    ave_lng = sum(p[1] for p in points)/len(points)
    #print(ave_lat, ave_lng)

    title = 'Intersection coordinates (X: {:10.2f}, Y: {:10.2f})\nDistance between towers: {:10.2f}m\n'.format(
             p3[0],p3[1],distances[0])+\
             'Distance {0} to intersection: {1:10.2f}m\n'.format(nomeA,distances[1])+\
             'Distance {0} to intersection: {1:10.2f}m\n'.format(nomeB,distances[2])
    
    mapa = folium.Map(
        location=[ave_lat, ave_lng],
        tiles='Stamen Terrain',
        zoom_start=13
    )
    #print(title)
    folium.Marker(
        location=g1,
        popup="<center>{}</center><br><center>{:.2f}-{:.2f}</center>".format(nomeA, g1[0], g1[1]),
        icon=folium.Icon(color='blue')
    ).add_to(mapa)
    folium.Marker(
        location=g2,
        popup="<center>{}</center><br><center>{:.2f}-{:.2f}</center>".format(nomeB, g2[0], g2[1]),
        icon=folium.Icon(color='blue')
    ).add_to(mapa)
    folium.Marker(
        location=g3,
        popup="<center>Interseção</center><br><center>{:.2f}-{:.2f}</center>".format(g3[0], g3[1]),
        icon=folium.Icon(color='red')
    ).add_to(mapa)

    folium.PolyLine([g1,g2], color="blue", weight=2.5, opacity=1).add_to(mapa)
    folium.PolyLine([g1,g3], color="green", weight=2.5, opacity=1).add_to(mapa)
    folium.PolyLine([g2,g3], color="green", weight=2.5, opacity=1).add_to(mapa)
        
    mapa.save("/content/mapa2.html")
    return(mapa)

def create_kml(z, p1, p2, p3, nomeA, nomeB, distances, azA, azB, northernHemisphere=False):
    lat, lng = utmToLatLng(z, p1[0], p1[1], northernHemisphere=False)
    g1 = (lng, lat)
    lat, lng = utmToLatLng(z, p2[0], p2[1], northernHemisphere=False)
    g2 = (lng, lat)
    lat, lng = utmToLatLng(z, p3[0], p3[1], northernHemisphere=False)
    g3 = (lng, lat)
    kml = simplekml.Kml(open=1)
    pntA = kml.newpoint(name=nomeA, description="Ponto de Observação A",
                   coords=[g1])  # lon, lat optional height
    pntB = kml.newpoint(name=nomeB, description="Ponto de Observação B",
                   coords=[g2])  # lon, lat optional height
    pntC = kml.newpoint(name="Cruzamento", description="Incêndio",
                   coords=[g3])  # lon, lat optional height 
    # Create a linestring that will hover 100m above the ground that is extended to the ground
    name1 = f'Linha de visada de A\nDistância: {distances[1]:.2f}m\nAzimute: {azA}{chr(176)}'
    linestring1 = kml.newlinestring(name=name1)
    linestring1.coords = [g1, g3]
    linestring1.altitudemode = simplekml.AltitudeMode.clamptoground
    linestring1.extrude = 1
    # Create a linestring that will hover 100m above the ground that is extended to the ground
    name2 = f'Linha de visada de B\nDistância: {distances[2]:.2f}m\nAzimute: {azB}{chr(176)}'
    linestring2 = kml.newlinestring(name=name2)
    linestring2.coords = [g2, g3]
    linestring2.altitudemode = simplekml.AltitudeMode.clamptoground
    linestring2.extrude = 1     
    kml.save('linhas_pontos.kml')     

def on_button_clicked(b):
  with output:
    distances, intersection = calcIntersection(xA,yA,azA,xB,yB,azB) 
    if not intersection.is_empty:
      #plotResult([xA,yA],[xB,yB],[intersection.x,intersection.y],nomeA,nomeB,distances)
      create_kml(23, [xA,yA],[xB,yB],[intersection.x,intersection.y],nomeA,nomeB,distances, azA, azB, northernHemisphere=False)
      display(plotMapFolium(23, [xA,yA],[xB,yB],[intersection.x,intersection.y],nomeA,nomeB,distances, northernHemisphere=False))
    else:
      print("There's no crossing point!")
  
button.on_click(on_button_clicked)
display(button, output)

Button(description='Calculate!', style=ButtonStyle())

Output()